<a href="https://colab.research.google.com/github/ShreeluSantosh/ShreeluSantosh/blob/main/docu3C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Uploading file to notebook

First, we need to upload the pdf file of the document to the Colab notebook, in order to be able to extract text

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Amicus Brief on Behalf of Mississippi, Alabama, Alaska, Arkansas etc....pdf to Amicus Brief on Behalf of Mississippi, Alabama, Alaska, Arkansas etc....pdf


# Step 2: Extracting text from pdf file

In [ ]:
import pandas as pd

In [ ]:
!pip install PyPDF2
import PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
pdf = open("/content/Amicus Brief on Behalf of Mississippi, Alabama, Alaska, Arkansas etc....pdf", "rb")
pdf_reader = PyPDF2.PdfReader(pdf)

In [ ]:
text = ""
text_list = {}
for i in range(7, 26):
  page_obj = pdf_reader.pages[i]
  text += page_obj.extract_text()
  text_list[i] = page_obj.extract_text().replace('\n', "")

In [ ]:
clean_text = text.replace("\n","")

In [ ]:
segments =  clean_text.split('.')
segments = [segment + '.' for segment in segments]
segments = [segment.split(',') for segment in segments]
segments = [item for sublist in segments for item in sublist]

In [ ]:
def create_sentences(segments, MIN_WORDS, MAX_WORDS):
  sentences = []
  is_new_sentence = True
  sentence_length = 0
  sentence_num = 0
  sentence_segments = []
  for i in range(len(segments)):
    if is_new_sentence == True:
      is_new_sentence = False
    sentence_segments.append(segments[i])
    segment_words = segments[i].split(' ')
    sentence_length += len(segment_words)
    if (sentence_length >= MIN_WORDS and segments[i][-1] == '.') or sentence_length >= MAX_WORDS:
      sentence = ' '.join(sentence_segments)
      sentences.append({
        'sentence_num': sentence_num,
        'text': sentence,
        'sentence_length': sentence_length
      })
      is_new_sentence = True
      sentence_length = 0
      sentence_segments = []
      sentence_num += 1
  return sentences

In [ ]:
def create_chunks(sentences, CHUNK_LENGTH, STRIDE):
  sentences_df = pd.DataFrame(sentences)
  chunks = []
  for i in range(0, len(sentences_df), (CHUNK_LENGTH - STRIDE)):
    chunk = sentences_df.iloc[i:i+CHUNK_LENGTH]
    chunk_text = ' '.join(chunk['text'].tolist())
    chunks.append({
      'start_sentence_num': chunk['sentence_num'].iloc[0],
      'end_sentence_num': chunk['sentence_num'].iloc[-1],
      'text': chunk_text,
      'num_words': len(chunk_text.split(' '))
    })
  chunks_df = pd.DataFrame(chunks)
  return chunks_df.to_dict('records')

In [ ]:
def parse_title_summary_results(results):
  out = []
  for e in results:
    e = e.replace('\n', '')
    if '|' in e:
      processed = {'title': e.split('|')[0],
                    'summary': e.split('|')[1][1:]
                    }
    elif ':' in e:
      processed = {'title': e.split(':')[0],
                    'summary': e.split(':')[1][1:]
                    }
    elif '-' in e:
      processed = {'title': e.split('-')[0],
                    'summary': e.split('-')[1][1:]
                    }
    else:
      processed = {'title': '',
                    'summary': e
                    }
    out.append(processed)
  return out

In [ ]:
sentences = create_sentences(segments, MIN_WORDS=20, MAX_WORDS=80)
chunks = create_chunks(sentences, CHUNK_LENGTH=5, STRIDE=1)
chunks_text = [chunk['text'] for chunk in chunks]

# Step 3: Setting up OpenAI

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
import openai
openai.organization = "{ORG}"
openai.api_key = "{YOUR_API_KEY}"

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(model=model, messages=messages, temperature=0.2)
    return response.choices[0].message["content"]

# Step 4: Getting Summary from the Document using HuggingFace

In [ ]:
import json
import requests
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
headers = {"Authorization": f"Bearer {HF_API_KEY}"}
def query(payload):
    data = json.dumps(payload)
    response = requests.request("POST", API_URL, headers=headers, data=data)
    return json.loads(response.content.decode("utf-8"))
summary = ""
sum_list = []
for i in chunks_text:
  data = query(
   {
        "inputs": i,
        "parameters": {"min_length":20,"do_sample": False},
    }
  )
  summary += data[0].get('summary_text')
  sum_list.append(data[0].get('summary_text'))

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving text.txt to text.txt


In [ ]:
f = open('text.txt', 'w')
f.write(summary)
f.close()
files.download('text.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Step 5: Writing Prompt for Extracting Important Items

In [ ]:
prompt = f"""
Extract 10 most important items needed\
by an attorney for a court trial\
from the summary delimited by triple backticks.\

Summary:```{summary}```
"""
response = get_completion(prompt)
print(response)

1. President Biden's executive order to expand access to abortion care
2. FDA's approval of mifepristone for chemically induced abortions
3. FDA's actions to establish a nationwide mail-order abortion-drug regime
4. Lawsuits seeking to enjoin and set aside the FDA's actions
5. FDA's violation of federal laws and regulations
6. FDA's abandonment of in-person dispensing requirements for mifepristone
7. Congress's consideration of narrowing statutes on abortion-inducing drugs
8. FDA's defiance of federal criminal law regarding abortion-inducing drugs
9. States' authority to legislate and regulate medical profession standards
10. The Biden Administration's encouragement of lawbreaking on a mass scale


# Step 6: Getting Page Number and Line Number for Each Item

In [ ]:
L = response.split('\n')
Sen = []
for i in L:
  Sen.append(i[3:])

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
for i in range(1, 11):
  ind = i-1
  doc1 = nlp(Sen[ind])
  max = 0
  m = 0
  page = 0
  sum_index = 0
  for j in sum_list:
    doc2 = nlp(j)
    if doc1.similarity(doc2) > max:
      max = doc1.similarity(doc2)
      sum_index  = sum_list.index(j)
  for k in text_list:
    doc3 = nlp(text_list[k])
    if nlp(sum_list[sum_index]).similarity(doc3) > m:
      m = nlp(sum_list[sum_index]).similarity(doc3)
      page = k
  p = 0
  l = 0
  for j in text_list[page].split('.'):
    line = nlp(j)
    if nlp(Sen[ind]).similarity(line) > p:
      p = nlp(Sen[ind]).similarity(line)
      l = text_list[page].index(j)
  print(i, '. ', Sen[ind], "[Page:", page, "line:", l,']')

1 .  President Biden's executive order to expand access to abortion care [Page: 21 line: 1061 ]
2 .  FDA's approval of mifepristone for chemically induced abortions [Page: 8 line: 417 ]
3 .  FDA's actions to establish a nationwide mail-order abortion-drug regime [Page: 11 line: 1040 ]
4 .  Lawsuits seeking to enjoin and set aside the FDA's actions [Page: 8 line: 1066 ]
5 .  FDA's violation of federal laws and regulations [Page: 21 line: 1061 ]
6 .  FDA's abandonment of in-person dispensing requirements for mifepristone [Page: 11 line: 1326 ]
7 .  Congress's consideration of narrowing statutes on abortion-inducing drugs [Page: 8 line: 955 ]
8 .  FDA's defiance of federal criminal law regarding abortion-inducing drugs [Page: 8 line: 955 ]
9 .  States' authority to legislate and regulate medical profession standards [Page: 21 line: 642 ]
10 .   The Biden Administration's encouragement of lawbreaking on a mass scale [Page: 13 line: 1240 ]
